## Scala Dataframes for Sparx

## Linear Regression

In [1]:
import org.apache.spark.ml.regression.GeneralizedLinearRegression

Intitializing Scala interpreter ...

Spark Web UI available at http://LAPTOP-B132RH0R.home:4042
SparkContext available as 'sc' (version = 3.3.0, master = local[*], app id = local-1668528934382)
SparkSession available as 'spark'


import org.apache.spark.ml.regression.GeneralizedLinearRegression


In [2]:
// Load training data
val dataset = spark.read.format("libsvm").load("data_files/sample_linear_regression_data.txt")

val glr = new GeneralizedLinearRegression()
  .setFamily("gaussian")
  .setLink("identity")
  .setMaxIter(10)
  .setRegParam(0.3)

// Fit the model
val model = glr.fit(dataset)

22/11/15 11:15:48 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.
22/11/15 11:15:55 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


dataset: org.apache.spark.sql.DataFrame = [label: double, features: vector]
glr: org.apache.spark.ml.regression.GeneralizedLinearRegression = glm_52f3d2addf79
model: org.apache.spark.ml.regression.GeneralizedLinearRegressionModel = GeneralizedLinearRegressionModel: uid=glm_52f3d2addf79, family=gaussian, link=identity, numFeatures=10


In [3]:
// Print the coefficients and intercept for generalized linear regression model
println(s"Coefficients: ${model.coefficients}")
println(s"Intercept: ${model.intercept}")

// Summarize the model over the training set and print out some metrics
val summary = model.summary
println(s"Coefficient Standard Errors: ${summary.coefficientStandardErrors.mkString(",")}")
println(s"T Values: ${summary.tValues.mkString(",")}")
println(s"P Values: ${summary.pValues.mkString(",")}")
println(s"Dispersion: ${summary.dispersion}")
println(s"Null Deviance: ${summary.nullDeviance}")
println(s"Residual Degree Of Freedom Null: ${summary.residualDegreeOfFreedomNull}")
println(s"Deviance: ${summary.deviance}")
println(s"Residual Degree Of Freedom: ${summary.residualDegreeOfFreedom}")
println(s"AIC: ${summary.aic}")
println("Deviance Residuals: ")
summary.residuals().show()

Coefficients: [0.010541828081257216,0.8003253100560949,-0.7845165541420371,2.3679887171421914,0.5010002089857577,1.1222351159753026,-0.2926824398623296,-0.49837174323213035,-0.6035797180675657,0.6725550067187461]
Intercept: 0.14592176145232041
Coefficient Standard Errors: 0.7950428434287478,0.8049713176546897,0.7975916824772489,0.8312649247659919,0.7945436200517938,0.8118992572197593,0.7919506385542777,0.7973378214726764,0.8300714999626418,0.7771333489686802,0.463930109648428
T Values: 0.013259446542269243,0.9942283563442594,-0.9836067393599172,2.848657084633759,0.6305509179635714,1.382234441029355,-0.3695715687490668,-0.6250446546128238,-0.7271418403049983,0.8654306337661122,0.31453393176593286
P Values: 0.989426199114056,0.32060241580811044,0.3257943227369877,0.004575078538306521,0.5286281628105467,0.16752945248679119,0.7118614002322872,0.5322327097421431,0.467486325282384,0.3872259825794293,0.753249430501097
Dispersion: 105.60988356821714
Null Deviance: 53229.3654338832
Residual Deg

summary: org.apache.spark.ml.regression.GeneralizedLinearRegressionTrainingSummary =
Coefficients:
    Feature Estimate Std Error T Value P Value
(Intercept)   0.1459    0.4639  0.3145  0.7532
 features_0   0.0105    0.7950  0.0133  0.9894
 features_1   0.8003    0.8050  0.9942  0.3206
 features_2  -0.7845    0.7976 -0.9836  0.3258
 features_3   2.3680    0.8313  2.8487  0.0046
 features_4   0.5010    0.7945  0.6306  0.5286
 features_5   1.1222    0.8119  1.3822  0.1675
 features_6  -0.2927    0.7920 -0.3696  0.7119
 features_7  -0.4984    0.7973 -0.6250  0.5322
 features_8  -0.6036    0.8301 -0.7271  0.4675
 features_9   0.6726    0.7771  0.8654  0.3872

(Dispersion parameter for gaussian family taken to be 105.6099)
    Null deviance: 53229.3654 on 490 degrees of freed...


In [6]:
model.predict(1.0)

<console>: 27: error: type mismatch;

## Classification

In [7]:
import org.apache.spark.ml.classification.LogisticRegression

import org.apache.spark.ml.classification.LogisticRegression


In [9]:
// Use Spark to read in the Titanic csv file.
val data = spark.read.option("header","true").option("inferSchema","true").format("csv").load("data_files/titanic.csv")

data: org.apache.spark.sql.DataFrame = [PassengerId: int, Survived: int ... 10 more fields]


In [10]:
data.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [11]:
// Print the Schema of the DataFrame
data.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [12]:
// Grab only the columns we want
val logregdataall = data.select(data("Survived").as("label"), $"Pclass", $"Sex", $"Age", $"SibSp", $"Parch", $"Fare", $"Embarked")

logregdataall: org.apache.spark.sql.DataFrame = [label: int, Pclass: int ... 6 more fields]


In [13]:
val logregdata = logregdataall.na.drop()

logregdata: org.apache.spark.sql.DataFrame = [label: int, Pclass: int ... 6 more fields]


In [14]:
import org.apache.spark.ml.feature.{VectorAssembler,StringIndexer,VectorIndexer,OneHotEncoder}
import org.apache.spark.ml.linalg.Vectors

import org.apache.spark.ml.feature.{VectorAssembler, StringIndexer, VectorIndexer, OneHotEncoder}
import org.apache.spark.ml.linalg.Vectors


In [15]:
// Deal with Categorical Columns
val genderIndexer = new StringIndexer().setInputCol("Sex").setOutputCol("SexIndex")
val embarkIndexer = new StringIndexer().setInputCol("Embarked").setOutputCol("EmbarkIndex")

val genderEncoder = new OneHotEncoder().setInputCol("SexIndex").setOutputCol("SexVec")
val embarkEncoder = new OneHotEncoder().setInputCol("EmbarkIndex").setOutputCol("EmbarkVec")

// Assemble everything together to be ("label","features") format
val assembler = (new VectorAssembler()
                  .setInputCols(Array("Pclass", "SexVec", "Age","SibSp","Parch","Fare","EmbarkVec"))
                  .setOutputCol("features") )

genderIndexer: org.apache.spark.ml.feature.StringIndexer = strIdx_3af4e4479d36
embarkIndexer: org.apache.spark.ml.feature.StringIndexer = strIdx_3cc1091c8d27
genderEncoder: org.apache.spark.ml.feature.OneHotEncoder = oneHotEncoder_88f82b3db852
embarkEncoder: org.apache.spark.ml.feature.OneHotEncoder = oneHotEncoder_b205339ef643
assembler: org.apache.spark.ml.feature.VectorAssembler = VectorAssembler: uid=vecAssembler_4cd65bb99eb5, handleInvalid=error, numInputCols=7


In [16]:
val Array(training, test) = logregdata.randomSplit(Array(0.7, 0.3), seed = 12345)

training: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [label: int, Pclass: int ... 6 more fields]
test: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [label: int, Pclass: int ... 6 more fields]


In [17]:
import org.apache.spark.ml.Pipeline
val lr = new LogisticRegression()
val pipeline = new Pipeline().setStages(Array(genderIndexer,embarkIndexer,genderEncoder,embarkEncoder,assembler, lr))

import org.apache.spark.ml.Pipeline
lr: org.apache.spark.ml.classification.LogisticRegression = logreg_3883d08a56fd
pipeline: org.apache.spark.ml.Pipeline = pipeline_15a1c8c50f58


In [18]:
val model = pipeline.fit(training)

model: org.apache.spark.ml.PipelineModel = pipeline_15a1c8c50f58


In [19]:
val results = model.transform(test)

results: org.apache.spark.sql.DataFrame = [label: int, Pclass: int ... 14 more fields]


In [20]:
// For Metrics and Evaluation
import org.apache.spark.mllib.evaluation.MulticlassMetrics
// Need to convert to RDD to use this
val predictionAndLabels = results.select($"prediction",$"label").as[(Double, Double)].rdd

import org.apache.spark.mllib.evaluation.MulticlassMetrics
predictionAndLabels: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[130] at rdd at <console>:32


In [21]:
// Instantiate metrics object
val metrics = new MulticlassMetrics(predictionAndLabels)

metrics: org.apache.spark.mllib.evaluation.MulticlassMetrics = org.apache.spark.mllib.evaluation.MulticlassMetrics@2ef83d54


In [22]:
// Confusion matrix
println("Confusion matrix:")
println(metrics.confusionMatrix)

Confusion matrix:
107.0  13.0  
18.0   62.0  
